In [1]:
import pandas as pd, re, numpy as np

n_pages = 2
with open("./output_text.txt", 'r') as f:
    text_data= f.readlines()
    lines = dict(lines = text_data)
    df = pd.DataFrame(lines).reset_index()

df['last_content'] = df['lines'].apply(
    lambda x: 1 if str(x).startswith("---") else None
)
df['is_date'] = df['lines'].str.strip().str.match(r'^\d{2}-\d{2}').astype(int)
df['solo_espacios'] = df['lines'].apply(lambda x: 1 if x.strip() == '' else None)
# 2. Detectar filas que contienen solo decimales con 2 dígitos
# df
df['lines'] = df['lines'].str.replace('\n', '')
# df.iloc[152:160][['lines']].to_clipboard()
df['solo_decimal'] = df['lines'].str.strip().str.match(r'\d{2}').astype(int)
df[['is_date', 'solo_espacios', 'solo_decimal']] = df[['is_date', 'solo_espacios', 'solo_decimal']].replace(0, np.nan)
df['last_content'] = df['last_content'].bfill()
# df['lines'] = df['lines'].apply(lambda x: re.sub(r'\s+', ' ', x))


# parte 2 columna adicional

df['is_last_col'] = ((df['last_content'] == 1) & (df['solo_decimal'] == 1)).astype(int)

# # # 3. Reemplazar los saltos de línea en la columna 'lines'
# df.to_csv("text.csv")
all = df.query('last_content == 1 & solo_decimal == 1 & is_last_col == 1')

In [2]:
cols = all[all['is_date'] == 1]
max_index1 = cols.index.max()
max_index1

np.int64(305)

In [3]:
df = all[all['index'] > max_index1]
min_index = df['index'].min()

# Encontrar el rango continuo de índices
min_index = df['index'].min()

# Encontrar el rango continuo de índices
max_index = min_index
for i in range(min_index, df['index'].max() + 1):
    if i not in df['index'].values:
        break
    max_index = i

# Filtrar el DataFrame para mantener solo el rango continuo
df_filtered = df[(df['index'] >= min_index) & (df['index'] <= max_index)].reset_index()
# pd.concat((cols['lines'], df_filtered['lines']), axis=1, ignore_index=True) 
# df_filtered


# result.to_csv('text.csv')



In [4]:
df_filtered

,level_0,index,lines,last_content,is_date,solo_espacios,solo_decimal,is_last_col
0,311,311,"69,769.36",1.0,NaN,NaN,1.0,1
1,312,312,"70,415.19",1.0,NaN,NaN,1.0,1
2,313,313,"71,070.44",1.0,NaN,NaN,1.0,1
3,314,314,"71,068.04",1.0,NaN,NaN,1.0,1
4,315,315,"71,713.87",1.0,NaN,NaN,1.0,1
5,316,316,"72,380.54",1.0,NaN,NaN,1.0,1
6,317,317,"73,130.54",1.0,NaN,NaN,1.0,1
7,318,318,"74,030.54",1.0,NaN,NaN,1.0,1
8,319,319,"75,859.71",1.0,NaN,NaN,1.0,1
9,320,320,"75,852.06",1.0,NaN,NaN,1.0,1


In [5]:
result = pd.DataFrame(
    {
        "cols": cols['lines'].values,
        "saldo": df_filtered['lines'].values
    }
)
def dividir_columna(texto):
    # Patrón para buscar VEN, INT o TLC
    patron = r'(.*?)\s+(VEN|CAJ|POS|TLC|INT|BPT|BPI)(.*)$'
    
    # Buscar el patrón en el texto
    match = re.search(patron, texto)
    
    if match:
        antes = match.group(1)
        codigo = match.group(2)
        despues = match.group(3)
        return pd.Series([antes, codigo, despues])
    else:
        # Si no hay coincidencia, devolver el texto original y valores vacíos
        return pd.Series([texto, '', ''])
result[['antes_codigo', 'codigo', 'despues_codigo']] = result['cols'].apply(dividir_columna)
result = result.drop(columns=['cols'], errors='ignore')
# result['despues_codigo']

df = result

def extraer_y_eliminar(texto):
    # 1. Patrón para encontrar el código (4 dígitos) y el monto juntos al final
    patron_completo = r'(\d{4})\s+([0-9,\.]+(?:\s*-)?)$'
    match_completo = re.search(patron_completo, texto.strip())
    
    if match_completo:
        codigo = match_completo.group(1)
        monto = match_completo.group(2).strip()
        
        # Encontrar dónde comienza el código + monto para eliminarlos
        inicio = texto.rfind(codigo)
        resto = texto[:inicio].strip()
        
        return pd.Series([resto, monto, codigo])
    
    # 2. Si no encuentra un código de 4 dígitos, buscar solo el monto
    patron_solo_monto = r'([0-9,\.]+(?:\s*-)?)$'
    match_solo_monto = re.search(patron_solo_monto, texto.strip())
    
    if match_solo_monto:
        monto = match_solo_monto.group(1).strip()
        
        # Encontrar dónde comienza el monto para eliminarlo
        inicio = texto.rfind(monto)
        resto = texto[:inicio].strip()
        
        return pd.Series([resto, monto, ""])
    
    # Si no se encuentra ningún patrón
    return pd.Series([texto, "", ""])

# Aplicar la función para extraer y eliminar
df[['despues_codigo_limpio', 'monto', 'codigo']] = df['despues_codigo'].apply(extraer_y_eliminar)

# Asegurarse de que los montos con signo negativo estén correctamente formateados
def corregir_formato_monto(monto):
    if not monto:
        return ""
    
    # Si termina con un guion, mover el guion al principio
    if monto.endswith('-'):
        # Quitar el guion del final
        monto_sin_guion = monto[:-1].strip()
        # Agregar el guion al principio
        return '-' + monto_sin_guion
    return monto

df['monto_formateado'] = df['monto'].apply(corregir_formato_monto)

# Mostrar el resultado
resultado = df[['despues_codigo', 'despues_codigo_limpio', 'monto', 'monto_formateado', 'codigo']]
# print(resultado)
# resultado[['despues_codigo_limpio']].to_clipboard()
result['despues_codigo'].to_clipboard()

ValueError: All arrays must be of the same length

In [141]:
result['antes_codigo']


0      02-05       EFECTIVO00000043693603
1      02-05       EFECTIVO00000070245924
2           02-05       COMIS.RECAUDACION
3                02-05       IMPUESTO ITF
4             06-05       A 191 2320322 0
5                06-05       IMPUESTO ITF
6      12-05       EFECTIVO00000070983945
7             12-05       A 193 2037242 0
8            12-05       A 191 94263671 0
9           12-05       COMIS.RECAUDACION
10               12-05       IMPUESTO ITF
11     13-05       EFECTIVO00020523078306
12          13-05       COMIS.RECAUDACION
13               13-05       IMPUESTO ITF
14     14-05       EFECTIVO00000044424853
15     14-05       EFECTIVO00000044424853
16          14-05       COMIS.RECAUDACION
17               15-05       IMPUESTO ITF
18     16-05       EFECTIVO00000070983945
19          16-05       COMIS.RECAUDACION
20               16-05       IMPUESTO ITF
21     18-05       EFECTIVO00000070983945
22          18-05       COMIS.RECAUDACION
23               18-05       IMPUE

In [128]:
r = resultado[['despues_codigo']]
def contar_tamano_string(texto):
    # Eliminar espacios al final
    texto_sin_espacios = texto.strip()
    
    # Contar longitud base
    longitud = len(texto_sin_espacios)
    
    # Si no termina con "-", añadir 1 al conteo
    if not texto_sin_espacios.endswith("-"):
        longitud += 1
    
    return longitud

r['len'] = r['despues_codigo'].apply(contar_tamano_string)
r
max_len = r['len'].max()
# r.query('len == @max_len').to_clipboard()
completo_texto = buscar_filas_con_espacios_pandas(r, 'despues_codigo')[['despues_codigo']]

C:\Users\JhonK\AppData\Local\Temp\ipykernel_26004\2421108739.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['len'] = r['despues_codigo'].apply(contar_tamano_string)
C:\Users\JhonK\AppData\Local\Temp\ipykernel_26004\4132873296.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_coincidencias'] = df[columna].apply(contar_coincidencias_espacios)


In [148]:
def detectar_posiciones_columnas(df, columna_origen):
    """
    Detecta las posiciones de inicio y fin de cada columna en un DataFrame
    con formato de tabla de ancho fijo.
    
    Args:
        df: DataFrame de pandas
        columna_origen: Nombre de la columna que contiene los datos en formato tabla
        
    Returns:
        Lista de tuplas (inicio, fin) para cada columna detectada
    """
    # Tomar una muestra de filas para analizar el patrón
    muestra = df[columna_origen].values
    
    # Buscar patrones de espacios para detectar columnas
    patrones = []
    for fila in muestra:
        # Encontrar secuencias de caracteres no espacios
        matches = re.finditer(r'\S+', fila)
        posiciones = [(m.start(), m.end()) for m in matches]
        patrones.append(posiciones)
    
    # Encontrar las columnas más consistentes
    inicio_columnas = {}
    fin_columnas = {}
    
    for posiciones in patrones:
        for i, (inicio, fin) in enumerate(posiciones):
            if i not in inicio_columnas:
                inicio_columnas[i] = []
            if i not in fin_columnas:
                fin_columnas[i] = []
            inicio_columnas[i].append(inicio)
            fin_columnas[i].append(fin)
    
    # Calcular las posiciones más frecuentes para cada columna
    posiciones_columnas = []
    for i in range(6):  # Sabemos que hay 6 columnas
        if i in inicio_columnas and i in fin_columnas:
            inicio = max(set(inicio_columnas[i]), key=inicio_columnas[i].count)
            fin = max(set(fin_columnas[i]), key=fin_columnas[i].count)
            posiciones_columnas.append((inicio, fin))
    
    return posiciones_columnas

def extraer_columnas(df, columna_origen, posiciones_columnas, nombres_columnas=None):
    """
    Extrae datos de una columna de texto con formato de tabla de ancho fijo
    y crea nuevas columnas en el DataFrame.
    
    Args:
        df: DataFrame de pandas
        columna_origen: Nombre de la columna que contiene los datos en formato tabla
        posiciones_columnas: Lista de tuplas (inicio, fin) para cada columna
        nombres_columnas: Lista de nombres para las nuevas columnas
        
    Returns:
        DataFrame con las nuevas columnas añadidas
    """
    if nombres_columnas is None:
        nombres_columnas = [f'columna_{i+1}' for i in range(len(posiciones_columnas))]
    
    df_resultado = df.copy()
    
    # Extraer cada columna usando las posiciones detectadas
    for i, (inicio, fin) in enumerate(posiciones_columnas):
        nombre = nombres_columnas[i]
        
        # Para la última columna, tomar hasta el final del string
        if i == len(posiciones_columnas) - 1:
            df_resultado[nombre] = df[columna_origen].str[inicio:].str.strip()
        else:
            df_resultado[nombre] = df[columna_origen].str[inicio:fin].str.strip()
    
    return df_resultado
posiciones_columnas = detectar_posiciones_columnas(completo_texto, 'despues_codigo')
# posiciones_columnas
nombres_columnas = ['codigo', 'numero', 'hora', 'identificador', 'cuenta', 'monto']

df_resultado = extraer_columnas(r, 'despues_codigo', posiciones_columnas, nombres_columnas)
df_resultado=df_resultado.drop(columns=['len', 'temp_coincidencias', 'despues_codigo'])

In [157]:
df_resultado['antes_codigo'] = result['antes_codigo']
df_resultado['saldo_contable'] = df_filtered['lines']
# df_resultado
df_resultado.to_excel('sample.xlsx')

In [ ]:
import pandas as pd
import re

def detectar_posiciones_columnas(df, columna_origen):
    """
    Detecta las posiciones de inicio y fin de cada columna en un DataFrame
    con formato de tabla de ancho fijo.
    
    Args:
        df: DataFrame de pandas
        columna_origen: Nombre de la columna que contiene los datos en formato tabla
        
    Returns:
        Lista de tuplas (inicio, fin) para cada columna detectada
    """
    # Tomar una muestra de filas para analizar el patrón
    muestra = df[columna_origen].values
    
    # Buscar patrones de espacios para detectar columnas
    patrones = []
    for fila in muestra:
        # Encontrar secuencias de caracteres no espacios
        matches = re.finditer(r'\S+', fila)
        posiciones = [(m.start(), m.end()) for m in matches]
        patrones.append(posiciones)
    
    # Encontrar las columnas más consistentes
    inicio_columnas = {}
    fin_columnas = {}
    
    for posiciones in patrones:
        for i, (inicio, fin) in enumerate(posiciones):
            if i not in inicio_columnas:
                inicio_columnas[i] = []
            if i not in fin_columnas:
                fin_columnas[i] = []
            inicio_columnas[i].append(inicio)
            fin_columnas[i].append(fin)
    
    # Calcular las posiciones más frecuentes para cada columna
    posiciones_columnas = []
    for i in range(6):  # Sabemos que hay 6 columnas
        if i in inicio_columnas and i in fin_columnas:
            inicio = max(set(inicio_columnas[i]), key=inicio_columnas[i].count)
            fin = max(set(fin_columnas[i]), key=fin_columnas[i].count)
            posiciones_columnas.append((inicio, fin))
    
    return posiciones_columnas

def extraer_columnas(df, columna_origen, posiciones_columnas, nombres_columnas=None):
    """
    Extrae datos de una columna de texto con formato de tabla de ancho fijo
    y crea nuevas columnas en el DataFrame.
    
    Args:
        df: DataFrame de pandas
        columna_origen: Nombre de la columna que contiene los datos en formato tabla
        posiciones_columnas: Lista de tuplas (inicio, fin) para cada columna
        nombres_columnas: Lista de nombres para las nuevas columnas
        
    Returns:
        DataFrame con las nuevas columnas añadidas
    """
    if nombres_columnas is None:
        nombres_columnas = [f'columna_{i+1}' for i in range(len(posiciones_columnas))]
    
    df_resultado = df.copy()
    
    for i, (inicio, fin) in enumerate(posiciones_columnas):
        nombre = nombres_columnas[i]
        df_resultado[nombre] = df[columna_origen].str[inicio:fin].str.strip()
    
    return df_resultado

# Ejemplo de uso con el DataFrame proporcionado
# Crear el DataFrame de ejemplo
datos = [
    "                    111-023   885158   11:09   BMOB91   1013        2,558.67     ",
    "                    111-023   887762   14:29   BMO668   1013        8,520.36     ",
    "                    111-008   007865   09:31   TNP267   4401        5,614.44-    ",
    "                    111-023   047490   09:39   BMO905   1013       15,530.80     ",
    "                    111-008   021443   11:18   TNP0Z5   4401          472.00-    ",
    "                    111-008   021437   11:18   TNP0XU   4401        1,455.36-    ",
    "                    111-023   442121   21:56   BMO323   1013        6,953.25     ",
    "                    111-023   690478   16:50   BMO938   1013       20,432.00     ",
    "                    111-023   481005   16:18   BMO788   1013       25,000.00     ",
    "                    111-023   722711   09:54   BMOC82   1013       18,930.00     ",
    "                    111-023   001088   09:18   BMO758   1013       18,810.00     ",
    "                    111-023   160686   19:32   BMO600   1013        1,939.57     "
]

df = pd.DataFrame(datos, columns=['despues_codigo'])

# Detectar posiciones de columnas
posiciones_columnas = detectar_posiciones_columnas(df, 'despues_codigo')

nombres_columnas = ['codigo', 'numero', 'hora', 'identificador', 'cuenta', 'monto']

df_resultado = extraer_columnas(df, 'despues_codigo', posiciones_columnas, nombres_columnas)

print("\nDataFrame con columnas extraídas:")
# df_resultado
posiciones_columnas

Posiciones detectadas de columnas (inicio, fin):
Columna 1: (20, 27)
Columna 2: (30, 36)
Columna 3: (39, 44)
Columna 4: (47, 53)
Columna 5: (56, 60)
Columna 6: (68, 76)

DataFrame con columnas extraídas:


[(20, 27), (30, 36), (39, 44), (47, 53), (56, 60), (68, 76)]

In [120]:
import pandas as pd

def buscar_filas_con_espacios_pandas(df, columna) -> pd.DataFrame:
    """
    Busca filas en un DataFrame que tengan exactamente 4 coincidencias
    de 3 espacios seguidos entre caracteres en la columna especificada.
    
    Args:
        df: DataFrame de pandas
        columna: Nombre de la columna a analizar
    
    Returns:
        DataFrame filtrado con las filas que cumplen el criterio
    """
    # Función para contar coincidencias de 3 espacios entre caracteres
    def contar_coincidencias_espacios(texto):
        texto = str(texto)
        contador_coincidencias = 0
        posicion_actual = 0
        
        while posicion_actual < len(texto) - 3:
            # Buscar 3 espacios seguidos
            if texto[posicion_actual:posicion_actual+3] == "   ":
                # Verificar que esté entre caracteres (no espacios)
                if (posicion_actual > 0 and texto[posicion_actual-1].strip() and
                    posicion_actual+3 < len(texto) and texto[posicion_actual+3].strip()):
                    contador_coincidencias += 1
            posicion_actual += 1
            
        return contador_coincidencias
    
    # Crear columna temporal con el conteo de coincidencias
    df['temp_coincidencias'] = df[columna].apply(contar_coincidencias_espacios)
    
    # Filtrar filas con exactamente 4 coincidencias
    resultado = df[df['temp_coincidencias'] == 4].copy()
    
    # Eliminar columna temporal
    resultado.drop('temp_coincidencias', axis=1, inplace=True)
    
    return resultado

# Ejemplo de uso
# Suponiendo que tu DataFrame se llama 'df' y la columna relevante 'texto'
# df_filtrado = buscar_filas_con_espacios_pandas(df, 'texto')

# Para probar con los datos de ejemplo
datos = [
    "                    111-023   885158   11:09   BMOB91   1013        2,558.67     ",
    "                    111-023   887762   14:29   BMO668   1013        8,520.36     ",
    "                    193-000   893438                    4983            7.00-    ",
    "                       -                                0909             .50-    ",
    "                    111-008   007865   09:31   TNP267   4401        5,614.44-    "
]

# Crear un DataFrame de ejemplo
df_ejemplo = pd.DataFrame(datos, columns=['texto'])
df_filtrado = buscar_filas_con_espacios_pandas(df_ejemplo, 'texto')
print(f"Se encontraron {len(df_filtrado)} filas con exactamente 4 coincidencias de 3 espacios entre caracteres:")
df_filtrado.to_clipboard()

Se encontraron 3 filas con exactamente 4 coincidencias de 3 espacios entre caracteres:


In [23]:
valores_coincides_napage = df.value_counts('lines').reset_index().query('count == @n_pages')

In [57]:
import pandas as pd
import re

# Datos de ejemplo
data = {
    'lines': [
        '82,996.31',
        '82,995.81',
        '77,381.37',
        '77,381.12',
        '92,911.92',
        '92,439.92',
        '90,984.56',
        '90,981.06'
    ]
}

# Crear DataFrame
df = pd.DataFrame(data)

# Función para verificar el formato usando regex
def tiene_formato_regex(valor):
    # Patrón regex para detectar un punto seguido de dos dígitos
    patron = r'\.\d{2}'
    partes = valor.split()
    if len(partes) > 1:
        numero = partes[1].replace(',', '')
        return bool(re.search(patron, numero))
    return False

# Aplicar la función y crear una nueva columna con 1 o 0
df['tiene_formato'] = df['lines'].apply(tiene_formato_regex).astype(int)

# Mostrar el DataFrame resultante
print(df)


       lines  tiene_formato
0  82,996.31              0
1  82,995.81              0
2  77,381.37              0
3  77,381.12              0
4  92,911.92              0
5  92,439.92              0
6  90,984.56              0
7  90,981.06              0
